In [3]:
# from tensorflow import keras
import keras
import numpy as np
import itertools
from random import randint
import json

Using TensorFlow backend.


In [2]:
# %%bash
# wget https://github.com/PizzaRollExpert/Rock-paper-scissors-data/raw/master/data.txt

In [4]:
data = open('data.txt').read()
data = data.split('\n')

#s represents rock, x scissors, p paper and - the end of a game.
#player is 1st move, algorithm 2nd
n_to_move = {
  1:'s',
  2:'p',
  3:'x'
}

full_n_to_move = {
  1:'rock',
  2:'paper',
  3:'scissors'
}

move_to_n = {
    's':1,
    'p':2,
    'x':3
}


def convert_data(data):
    result = []
    game = []
    for row in data:
        moves = []
        if row == '-':
            if len(game) > 2:
                result.append(np.array(game))
                game = []
            continue
        for move in row:
            moves.append(move_to_n[move])
        if len(moves) > 1:
            game.append(np.array(moves))
        moves = []
    return np.array(result)

data = convert_data(data)
data2 = np.array([np.flip(games, 1) for games in data]) #reverse for player2
data = np.concatenate((data2, data))

def calculate_winner(x):
    result = x[0] - x[1]
    if result == 0: return 0 #tie
    if result in (1,-2): return 1 #win
    return -1 #lose

def create_xy_winner(data):
    result_y = []
    result_x = []
    for game in data:
        game_y = []
        score = 0
        for i, moves in enumerate(game):
            if i+1 == len(game):
                result_y.append(np.array(game_y))
                result_x.append(game[:-1]) #remove last moves from x
                #skip last game as we dont know what player1 will choose next move
                continue
            score += calculate_winner(game[i])
            game_y.append([game[i+1][0], score]) #append next player1 move
    return np.array(result_x), np.array(result_y)

X_gh, y_gh = create_xy_winner(data)

FileNotFoundError: [Errno 2] No such file or directory: 'data.txt'

In [4]:
#RUN THIS TO TRAIN ON COLLECTED DATA

collected_data = open('collected-data.txt').read().split('\n')
collected_parsed = [json.loads(row) for row in collected_data[:-1]]
collected_data = []
user_games = {}
for i in range(len(collected_parsed)):
    user_games[collected_parsed[i][0]] = collected_parsed[i][1]
for collected in user_games.values():
    split_games = [collected[x:x+30] for x in range(0, len(user_games), 28) if (len(collected) - x > 5)]
    for games in split_games:
        collected_data.append(np.array([[np.argmax(game[:4]), np.argmax(game[4:])] for game in games]))

collected_data = np.array(collected_data)

X_col, y_col = create_xy_winner(collected_data)

In [9]:
main_input = keras.Input(shape=(None,8), dtype='float32')
dense = keras.layers.Dense(32, activation='relu')(main_input)
lstm = keras.layers.LSTM(96, return_sequences=True)(dense)
main_output = keras.layers.Dense(20, activation='relu')(lstm)
main_output = keras.layers.Dense(4, activation='softmax')(main_output)
second_output = keras.layers.Dense(20, activation='tanh')(lstm)
second_output = keras.layers.Dense(1)(second_output)
model = keras.Model(inputs=[main_input,], outputs=[main_output, second_output])

opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss=['categorical_crossentropy', 'mse'], optimizer=opt, metrics=['accuracy'], loss_weights=[1., 0.1])

print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, 8)      0                                            
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, None, 32)     288         input_2[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, None, 96)     49536       dense_6[0][0]                    
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, None, 20)     1940        lstm_2[0][0]                     
__________________________________________________________________________________________________
dense_9 (D

In [10]:
#MODEL FIT

for (X, y, lr) in [(X_col, y_col, 0.001), (X_gh, y_gh, 0.0008), (X_col, y_col, 0.0007), (X_gh, y_gh, 0.00005), (X_col, y_col, 0.0001), (X_col, y_col, 0.00001)]:
    ## Anneal learning late
    model.optimizer.lr.assign(lr)
    print(f'**{lr}**\n')
    batch_x = []
    batch_y = []
    for i in range(len(X)):
        X_, y_ = X[i], y[i]
        X_ = keras.utils.to_categorical(X_, 4)
        X_ = X_.reshape(1, X_.shape[0], 8)
        y_1 = keras.utils.to_categorical(y_[:,0].reshape(1, y_.shape[0], 1), 4)
        y_2 = y_[:,1].reshape(1, y_.shape[0], 1)
        verbose = 2 if (i % 15 == 0) else 0 # print only 1/25th of the time
        model.fit(X_, [y_1, y_2], epochs=1, shuffle=False, batch_size=1, verbose=verbose)


**0.001**

Epoch 1/1
 - 1s - loss: 1.6223 - dense_8_loss: 1.3738 - dense_10_loss: 2.4852 - dense_8_acc: 0.3333 - dense_10_acc: 0.2222
Epoch 1/1
 - 0s - loss: 1.9604 - dense_8_loss: 1.3541 - dense_10_loss: 6.0638 - dense_8_acc: 0.3333 - dense_10_acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.4683 - dense_8_loss: 1.5834 - dense_10_loss: 18.8486 - dense_8_acc: 0.2632 - dense_10_acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 1.4165 - dense_8_loss: 1.2081 - dense_10_loss: 2.0847 - dense_8_acc: 0.6667 - dense_10_acc: 0.1667
Epoch 1/1
 - 0s - loss: 1.4048 - dense_8_loss: 1.1640 - dense_10_loss: 2.4083 - dense_8_acc: 0.5185 - dense_10_acc: 0.2222
Epoch 1/1
 - 0s - loss: 2.4434 - dense_8_loss: 1.2005 - dense_10_loss: 12.4294 - dense_8_acc: 0.4483 - dense_10_acc: 0.0345
Epoch 1/1
 - 0s - loss: 1.3356 - dense_8_loss: 0.6226 - dense_10_loss: 7.1301 - dense_8_acc: 0.7857 - dense_10_acc: 0.2143
Epoch 1/1
 - 0s - loss: 2.0448 - dense_8_loss: 1.0985 - dense_10_loss: 9.4634 - dense_8_acc: 0.3333 - dense_10_acc: 0.

In [7]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, 'Full-Model')

In [8]:
%%bash
rm -rf static/Full-Model
mv Full-Model static/